# Fetching the city districts' names with postal codes

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Let's start with Chicago

In [571]:
url = 'https://www.seechicagorealestate.com/chicago-zip-codes-by-neighborhood.php'
raw = requests.get(url).text

In [573]:
table = BeautifulSoup(raw, 'html.parser').find('table')

In [574]:
df = pd.read_html(str(table))[0]

In [575]:
df.columns = ['name', 'zip']

In [576]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

There are 77 neighborhoods in Chicago and comparable number of them in other two cities of interest (LA and NY). We don't need such level of fragmentation for our analysis, so in the case of Chicago 

In [577]:
side = ''
for i in df.index:
    if df.at[i, 'zip'] == 'Zip Code':
        side = df.at[i, 'name']
    df.at[i, 'city_district'] = side

In [578]:
df.head()

,name,zip,city_district
0,Downtown,Zip Code,Downtown
1,Cathedral District,60611,Downtown
2,Central Station,60605,Downtown
3,Dearborn Park,60605,Downtown
4,Gold Coast,"60610, 60611",Downtown


In [579]:
# df['zip'] = df.zip.apply(lambda x: x.split(sep=','))

In [580]:
df.city_district.value_counts()

Northwest Side     56
Western Suburbs    33
Downtown           25
South Side         24
North Side         22
North Shore        17
Far North Side     15
Name: city_district, dtype: int64

In [581]:
for i in df.index:
    if df.at[i, 'zip'] == 'Zip Code':
        df.drop(i, inplace=True)

In [582]:
df.drop(df[df.zip == 'Zip Code'].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [584]:
df.sample(5)

,name,zip,city_district
12,Printers Row,60605,Downtown
125,Sleepy Hollow,60118,Northwest Side
134,Burr Ridge,60527,Western Suburbs
137,Elmhurst,60126,Western Suburbs
31,Lakeview East,"60613, 60657",North Side


In [587]:
df = df[['city_district', 'zip']]

In [589]:
df.head()

,city_district,zip
0,Downtown,60611
1,Downtown,60605
2,Downtown,60605
3,Downtown,"60610, 60611"
4,Downtown,"60601, 60602, 60603, 60604, 60605, 60606, 6060..."


Let's group Chicago zip codes by city districts:

In [591]:
df = df.groupby(['city_district'])['zip'].apply(', '.join).reset_index()

In [592]:
df

,city_district,zip
0,Downtown,"60611, 60605, 60605, 60610, 60611, 60601, 6060..."
1,Far North Side,"60640, 60625, 60613, 60640, 60613, 60640, 6064..."
2,North Shore,"60015, 60015, 60201, 60202, 60203, 60204, 6020..."
3,North Side,"60613, 60657, 60613, 60657, 60614, 60622, 6064..."
4,Northwest Side,"60625, 60618, 60639, 60641, 60630, 60618, 6063..."
5,South Side,"60609, 60620, 60643, 60655, 60608, 60609, 6061..."
6,Western Suburbs,"60126, 60502, 60503, 60504, 60505, 60506, 6050..."


The things are a bit complicated when the city of LA is concerned. I did not find a web page where all the city's districts neighborhoods and zip codes were in one place, so I have to compile my own table from different sources. Let's first load a csv file with neighborhoods and districts. Later we'll add there corresponding zip codes.

In [14]:
la = pd.read_csv('la.csv', delimiter=',')

In [15]:
la.head()

,name,city_district
0,Bunker Hill,Downtown Los Angeles
1,Chinatown,Downtown Los Angeles
2,Civic Center,Downtown Los Angeles
3,Fashion District,Downtown Los Angeles
4,Financial District,Downtown Los Angeles


In [16]:
la['name'] = la.name.apply(lambda x: x.strip())

Let's fetch the table with LA county cities/communities and and their zip codes. 

In [17]:
url = 'http://www.laalmanac.com/communications/cm02_communities.php'
raw = requests.get(url).text
table = BeautifulSoup(raw, 'html.parser').find('table')

In [18]:
la2 = pd.read_html(str(table))[0]

In [19]:
la2.sample(10)

,City/Community,Zip Code(s)
4,Alhambra,"91801, 91803"
334,Los Angeles (Terminal Island),90731
63,Carson,"90745, 90746, 90810"
218,Long Beach (AARP),90847
412,Montebello,90640
126,Echo Park (Los Angeles),90026
339,Los Angeles (Tujunga),91042
571,"Two Harbors, Santa Catalina Island (PO Boxes)",90704
185,La Cañada Flintridge,91011
142,Fuller Theological Seminary (Pasadena),91182


In [20]:
la2.columns = ['name', 'zip']

First we need to extract the data related only to the city of LA:

In [21]:
la2 = la2[la2.name.str.contains('\(Los Angeles\)')]

In [22]:
la2.sample()

,name,zip
640,Woodland Hills (Los Angeles),"91364, 91367"


In [23]:
# getting rid of the text in parentheses:
la2['name'] = la2.name.str.split('(').str.get(0)

In [24]:
la2.sample(5)

,name,zip
421,Mt. Washington,90065
553,Sylmar,91392
175,Hyde Park,90043
613,West Los Angeles,90025
632,Wilshire Center,"90070, 90075, 90076"


In [25]:
la2[la2.name.str.contains('/') | la2.name.str.contains(',')]

,name,zip
25,"Bel Air Estates, Beverly Glen",90077
36,"Beverly Glen, Bel Air Estates",90077
132,"El Sereno, Monterey Hills, University Hills",90032
141,"Florence-Graham, South Los Angeles",90001
161,Harbor Gateway/Shoestring,"90061, 90247, 90248, 90501, 90502"
211,"Lincoln Heights, Montecito Hieghts",90031
385,"Los Angeles International Airport, Westchester",90045
413,"Montecito Heights, Lincoln Heights",90031
414,"Monterey Hills, El Sereno, University Hills",90032
433,Northridge/JB Lansing Co,91329


If the district name contains slash, everything after the slash is just a refinement that can be omitted:

In [26]:
for i in la2[la2.name.str.contains('/', na=False)].index:
    la2.at[i, 'name'] = la2.at[i, 'name'].split('/')[0]

In the case of comma presence in `name` column, we need to divide it into separate entries with the same `zip` number:

In [27]:
tmp = pd.DataFrame(columns=['name', 'zip'])
for i in la2.index:
    for j in la2.name[i].split(','):
        tmp.loc[len(tmp)] = [j.strip(), la2.zip[i]]

In [28]:
tmp1 = pd.DataFrame(columns=['name', 'zip'])
for i in tmp.index:
    for j in tmp.zip[i].split(','):
        tmp1.loc[len(tmp1)] = [tmp.name[i], j.strip()]

In [29]:
tmp1.drop_duplicates(inplace=True)

In [ ]:
# merging the tables...
la_final = pd.merge(left=tmp1, right=la, on='name', how='left')

In [366]:
la_final.groupby(['city_district'])['zip'].apply(', '.join).reset_index()

,city_district,zip
0,Downtown Los Angeles,"90012, 90030, 90086, 90087, 90012, 90014, 9007..."
1,East Los Angeles,"90039, 90033, 90063, 90065, 90041, 90032, 9003..."
2,Greater Hollywood,"90029, 90027, 90028, 90038, 90046, 90068, 9006..."
3,Harbor Area,"90710, 90061, 90247, 90248, 90501, 90502, 9073..."
4,Mid-City Los Angeles,"90019, 90004, 90005, 90020, 90006, 90005, 9000..."
5,North West Los Angeles,"90077, 90049, 90067, 90064, 90272, 90064, 9002..."
6,Rampart Area,"90026, 90026, 90039, 90004, 90057"
7,San Fernando Valley,"91331, 91303, 91304, 91305, 91308, 91309, 9131..."
8,South East Los Angeles,"90002, 90003, 90007, 90011, 90037, 90058"
9,South Los Angeles,"90008, 90008, 90001, 90043, 90018, 90008, 9001..."


In [548]:
# backing up...
la_final.groupby(['city_district'])['zip'].apply(', '.join).reset_index().to_csv('la_grouped.csv', index=False)

In [546]:
la_grouped = la_final.groupby(['city_district'])['zip'].apply(', '.join).reset_index()

In [556]:
la_grouped

,city_district,zip
0,Downtown Los Angeles,"90012, 90030, 90086, 90087, 90012, 90014, 9007..."
1,East Los Angeles,"90039, 90033, 90063, 90065, 90041, 90032, 9003..."
2,Greater Hollywood,"90029, 90027, 90028, 90038, 90046, 90068, 9006..."
3,Harbor Area,"90710, 90061, 90247, 90248, 90501, 90502, 9073..."
4,Mid-City Los Angeles,"90019, 90004, 90005, 90020, 90006, 90005, 9000..."
5,North West Los Angeles,"90077, 90049, 90067, 90064, 90272, 90064, 9002..."
6,Rampart Area,"90026, 90026, 90039, 90004, 90057"
7,San Fernando Valley,"91331, 91303, 91304, 91305, 91308, 91309, 9131..."
8,South East Los Angeles,"90002, 90003, 90007, 90011, 90037, 90058"
9,South Los Angeles,"90008, 90008, 90001, 90043, 90018, 90008, 9001..."


In [593]:
df = df.append(la_grouped)

In [594]:
df

,city_district,zip
0,Downtown,"60611, 60605, 60605, 60610, 60611, 60601, 6060..."
1,Far North Side,"60640, 60625, 60613, 60640, 60613, 60640, 6064..."
2,North Shore,"60015, 60015, 60201, 60202, 60203, 60204, 6020..."
3,North Side,"60613, 60657, 60613, 60657, 60614, 60622, 6064..."
4,Northwest Side,"60625, 60618, 60639, 60641, 60630, 60618, 6063..."
5,South Side,"60609, 60620, 60643, 60655, 60608, 60609, 6061..."
6,Western Suburbs,"60126, 60502, 60503, 60504, 60505, 60506, 6050..."
0,Downtown Los Angeles,"90012, 90030, 90086, 90087, 90012, 90014, 9007..."
1,East Los Angeles,"90039, 90033, 90063, 90065, 90041, 90032, 9003..."
2,Greater Hollywood,"90029, 90027, 90028, 90038, 90046, 90068, 9006..."


The final step here -- fetching NYC boroughs with appropriate zip code groups:

In [595]:
url = 'https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm'
raw = requests.get(url).text

In [596]:
# table = BeautifulSoup(raw, 'html.parser').find('table', {'class': 'wikitable sortable'})

In [597]:
table = BeautifulSoup(raw, 'html.parser').find('table')

In [598]:
ny = pd.read_html(str(table))[0]

In [600]:
ny.head()

,Borough,Neighborhood,ZIP Codes
0,Bronx,Central Bronx,"10453, 10457, 10460"
1,Bronx,Bronx Park and Fordham,"10458, 10467, 10468"
2,Bronx,High Bridge and Morrisania,"10451, 10452, 10456"
3,Bronx,Hunts Point and Mott Haven,"10454, 10455, 10459, 10474"
4,Bronx,Kingsbridge and Riverdale,"10463, 10471"


Fortunatelly, there is nothing much to change here except removing the`neighborhood` column and grouping the zip codes by boroughs:

In [604]:
ny.rename({'Borough':'city_district', 'ZIP Codes':'zip'}, axis=1, inplace=True)

In [606]:
ny_final = ny.drop('Neighborhood', axis=1)

In [608]:
ny_final.head()

,city_district,zip
0,Bronx,"10453, 10457, 10460"
1,Bronx,"10458, 10467, 10468"
2,Bronx,"10451, 10452, 10456"
3,Bronx,"10454, 10455, 10459, 10474"
4,Bronx,"10463, 10471"


In [610]:
# final grouping...
ny_final = ny_final.groupby(['city_district'])['zip'].apply(', '.join).reset_index()

In [611]:
ny_final

,city_district,zip
0,Bronx,"10453, 10457, 10460, 10458, 10467, 10468, 1045..."
1,Brooklyn,"11212, 11213, 11216, 11233, 11238, 11209, 1121..."
2,Manhattan,"10026, 10027, 10030, 10037, 10039, 10001, 1001..."
3,Queens,"11361, 11362, 11363, 11364, 11354, 11355, 1135..."
4,Staten Island,"10302, 10303, 10310, 10306, 10307, 10308, 1030..."


In [612]:
df = df.append(ny_final)

In [614]:
df.reset_index(drop=True, inplace=True)

## The final result:

In [615]:
df

,city_district,zip
0,Downtown,"60611, 60605, 60605, 60610, 60611, 60601, 6060..."
1,Far North Side,"60640, 60625, 60613, 60640, 60613, 60640, 6064..."
2,North Shore,"60015, 60015, 60201, 60202, 60203, 60204, 6020..."
3,North Side,"60613, 60657, 60613, 60657, 60614, 60622, 6064..."
4,Northwest Side,"60625, 60618, 60639, 60641, 60630, 60618, 6063..."
5,South Side,"60609, 60620, 60643, 60655, 60608, 60609, 6061..."
6,Western Suburbs,"60126, 60502, 60503, 60504, 60505, 60506, 6050..."
7,Downtown Los Angeles,"90012, 90030, 90086, 90087, 90012, 90014, 9007..."
8,East Los Angeles,"90039, 90033, 90063, 90065, 90041, 90032, 9003..."
9,Greater Hollywood,"90029, 90027, 90028, 90038, 90046, 90068, 9006..."


Let's save the table above into a csv file for future use in our main project:

In [616]:
df.to_csv('zip_codes.csv', index=False) 